In [1]:
!pip install transformers datasets evaluate pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cufft-cu12==1

In [11]:
import pandas as pd
import evaluate
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
import evaluate


In [12]:
df = pd.read_csv("/kaggle/input/datasetfootball/FinalDataset.csv")

In [8]:
tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-arxiv")

OSError: There was a specific connection error when trying to load google/bigbird-pegasus-large-arxiv:
500 Server Error: Internal Server Error for url: https://huggingface.co/google/bigbird-pegasus-large-arxiv/resolve/main/config.json (Request ID: Root=1-67fdf700-12164c1a3f66d7a45b8b145b;beda5234-4249-4961-b02d-71c479c9897c)

Internal Error - We're working hard to fix this as soon as possible!

In [ ]:
def is_within_limit(text):
    tokens = tokenizer(text, return_tensors="pt", truncation=False)["input_ids"][0]
    return len(tokens) <= 4096

df = df[df["Commentary"].apply(is_within_limit)].reset_index(drop=True)
print(f"Remaining samples after filtering: {len(df)}")

In [ ]:
dataset = Dataset.from_pandas(df)

In [ ]:
dataset = dataset.train_test_split(test_size=0.1)
train_data = dataset["train"]
val_data = dataset["test"]

In [ ]:
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["Commentary"],
        max_length=4096,
        padding="max_length",
        truncation=True
    )
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["MatchReport"],
            max_length=1024,
            padding="max_length",
            truncation=True
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_val = val_data.map(preprocess_function, batched=True)


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/bigbird-pegasus-large-arxiv")

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./bigbird-football-model",
    eval_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=1,  # BigBird is large, 1 may be safest
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    logging_steps=10
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    return rouge.compute(predictions=decoded_preds, references=decoded_labels)